In [1]:
abstract GradFunction
abstract ProxFunction

# loss functions
type SquareLoss <: GradFunction
end

function grad(::SquareLoss, X, y, theta)
    return 2*(X*X'*theta - X*y)
end
function loss_eval(::SquareLoss, X, y, theta)
    return norm(X'*theta - y)^2
end

square_loss = SquareLoss()


# regularizers
type SquareReg <: ProxFunction
end

function prox(::SquareReg, t, z)
    return z/(t+1)
end

square_reg = SquareReg()

type TrivialReg <: ProxFunction
end

function prox(::TrivialReg, t, z)
    return z
end

trivial_reg = TrivialReg()

TrivialReg()

In [2]:
# Empirical risk minimization problem
type Erm
    X::Array{Float64, 2}
    y::Array{Float64, 1}
    loss::GradFunction
    reg::ProxFunction
    opt_val::Float64
    opt_x::Array{Float64, 1}
end

# Constructor

# without regularizer
function Erm(X::Array{Float64, 2}, y::Array{Float64, 1},
        loss::GradFunction)
    return Erm(X, y, loss, trivial_reg, 0, zeros(size(X, 1)))
end

# with regularizer
function Erm(X::Array{Float64, 2}, y::Array{Float64, 1},
        loss::GradFunction, reg::ProxFunction)
    return Erm(X, y, loss, reg, 0, zeros(size(X, 1)))
end

function solve!(erm::Erm, x0=nothing, tol=1e-5, beta=.5, max_iter=300, verbose=true)
    converged = false
    
    if x0 == nothing
        x0 = zeros(size(erm.X, 1))
    end
    
    x = x0
    prev_eval = nothing
    
    for i in 1:max_iter
        lambda = 1
        curr_grad = grad(erm.loss, erm.X, erm.y, x)
        curr_eval = loss_eval(erm.loss, erm.X, erm.y, x)
        
        # Prox iteration
        while true
            z = prox(erm.reg, lambda, x - lambda*curr_grad)
            delta = z-x
            z_loss = loss_eval(erm.loss, erm.X, erm.y, z)
            if z_loss <= curr_eval + dot(curr_grad,delta) + 1/(2*lambda)*norm(delta)^2
                x = z
                break
            end
            lambda *= beta
        end
        
        if prev_eval != nothing && abs(prev_eval - curr_eval) < tol
            converged = true
            if verbose
                info("Converged after $(i) iterations")
            end
            break
        end
        
        prev_eval = curr_eval
    end
    if !converged
        warn("Failed to converge after $(max_iter) iterations")
    end
    
    erm.opt_x = x
    erm.opt_val = loss_eval(erm.loss, erm.X, erm.y, x)
end

solve! (generic function with 6 methods)

In [3]:
X = rand(4, 4)
y = rand(4)
erm = Erm(X, y, square_loss, trivial_reg)

Erm([0.903013 0.725972 0.735948 0.384269; 0.104861 0.757777 0.195637 0.639418; 0.800697 0.905801 0.95659 0.117294; 0.0714895 0.913814 0.2639 0.622284],[0.03035,0.2648,0.226023,0.353814],SquareLoss(),TrivialReg(),0.0,[0.0,0.0,0.0,0.0])

In [4]:
solve!(erm)

INFO: Converged after 45 iterations


0.028214373155633767

In [5]:
norm(X'*(pinv(X')*y) - y)^2

2.669801126107362e-29

In [6]:
pinv(X')*y

4-element Array{Float64,1}:
 -2.81509
  9.70723
  2.67097
 -8.17103

In [7]:
erm.opt_x

4-element Array{Float64,1}:
  0.106918
  0.235819
 -0.054829
  0.153149

In [8]:
loss_eval(square_loss, erm.X, erm.y, erm.opt_x)

0.028214373155633767